In [ ]:
!pip install tensorflow==1.8

ERROR: Could not find a version that satisfies the requirement tensorflow==1.8 (from versions: 1.13.0rc1, 1.13.0rc2, 1.13.1, 1.13.2, 1.14.0rc0, 1.14.0rc1, 1.14.0, 1.15.0rc0, 1.15.0rc1, 1.15.0rc2, 1.15.0rc3, 1.15.0, 1.15.2, 1.15.3, 2.0.0a0, 2.0.0b0, 2.0.0b1, 2.0.0rc0, 2.0.0rc1, 2.0.0rc2, 2.0.0, 2.0.1, 2.0.2, 2.1.0rc0, 2.1.0rc1, 2.1.0rc2, 2.1.0, 2.1.1, 2.2.0rc0, 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0)
ERROR: No matching distribution found for tensorflow==1.8
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_similarity_score

Now time for the most exciting part, the actual modeling

In [0]:
df = pd.read_csv("tokenized.csv")
targets = df[["initial_indice","final_indice"]]
targets.head()

In [0]:
training = pd.read_csv("pad_token_text.csv",header= None)
training.head()

We will now split the data into a training and test set. The test set will not be used for modeling and will be used to evaluate the models on data they have not seen before.

In [0]:
x_train, x_test, y_train, y_test = train_test_split(training.values, targets.values, test_size=0.2, random_state=42)

In [0]:
def Baseline(vocab_size):
    model = Sequential([
        Embedding(vocab_size, 128, input_length=33),
        Bidirectional(GRU(128, return_sequences=True, dropout=0.8, recurrent_dropout=0.8)),
        Bidirectional(GRU(128,return_sequences=True, dropout=0.8, recurrent_dropout=0.8)),
        BatchNormalization(),
        Dense(64, activation='elu',kernel_regularizer=l1_l2()),
        Dropout(0.8),
        Dense(2, activation='elu'),
        Flatten(),
        Dense(2, activation='elu')

    ])
    return model

In [0]:
xtrain.shape

In [0]:
ytrain.shape

In [0]:
vocab = 20000
model = Baseline(vocab)
es = EarlyStopping(patience=5)
mcp_save = ModelCheckpoint('tweet_sentiment.hdf5', save_best_only=True, monitor='val_mse')
model.compile(loss="mse",optimizer="adam",metrics=['mse',"mae"])
model.summary()


In [0]:
history = model.fit(x=xtrain, y=ytrain, batch_size = 32, epochs=100, validation_split = 0.2,callbacks=[es,mcp_save])

In [0]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [0]:
plot_graphs(history,"mse")


In [0]:
def jaccard_in(y_true, y_pred):
    true= range(start=y_true[0], stop=y_true[1])
    pred= range(start=y_pred[0], stop=y_pred[1])
    return jaccard_similarity_score(true, pred)
def jaccard(y_true, y_pred):
    return

